The source that got the data from: https://drive.google.com/drive/folders/1crEAKkiZg60oNKK37WzfpRo_M6E1MXSz

In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    try:
        # Connect to the default database (e.g., 'postgres')
        connect = psycopg2.connect('host=localhost port=5432 dbname=postgres user=postgres password=your_pass')
        connect.autocommit = True  # Set autocommit to True for database creation

        # Create a curs object
        cursor = connect.cursor()

        # Execute the CREATE DATABASE statement
        cursor.execute('DROP DATABASE IF EXISTS WEALTH')
        cursor.execute('CREATE DATABASE WEALTH')

        connect.close()
        connect = psycopg2.connect('host=localhost port=5432 dbname=wealth user=postgres password=your_pass')
        cursor = connect.cursor()


        return cursor, connect

    except psycopg2.Error as e:
        print("Error: Unable to create the database")
        print(e)
        return None, None


In [3]:
def drop_tables(cursor, connect):
    for query in drop_table_query:
        cursor.execute(query)
        connect.commit()

In [4]:
def create_tables(cursor, connect):
    for query in create_table_query:
        cursor.execute(query)
        connect.commit()

In [5]:
accountscountry = pd.read_csv('Wealth-AccountsCountry.csv')

In [6]:
accountscountry.head(5)

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [7]:
accountscountry.columns.unique()

Index(['Code', 'Long Name', 'Income Group', 'Region', 'Lending category',
       'Other groups', 'Currency Unit', 'Latest population census',
       'Latest household survey', 'Special Notes',
       'National accounts base year', 'National accounts reference year',
       'System of National Accounts', 'SNA price valuation',
       'Alternative conversion factor', 'PPP survey years',
       'Balance of Payments Manual in use', 'External debt Reporting status',
       'System of trade', 'Government Accounting concept',
       'IMF data dissemination standard',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', '2-alpha code', 'WB-2 code',
       'Table Name', 'Short Name'],
      dtype='object')

In [8]:
accountscountry_clean = accountscountry[['Code', 'Long Name','Currency Unit','Table Name', 'Short Name']]

In [9]:
accountscountry_clean

,Code,Long Name,Currency Unit,Table Name,Short Name
0,ALB,Republic of Albania,Albanian lek,Albania,Albania
1,ARG,Argentine Republic,Argentine peso,Argentina,Argentina
2,ARM,Republic of Armenia,Armenian dram,Armenia,Armenia
3,AUS,Commonwealth of Australia,Australian dollar,Australia,Australia
4,AUT,Republic of Austria,Euro,Austria,Austria
...,...,...,...,...,...
7592,NaN,NaN,NaN,NaN,NaN
7593,NaN,NaN,NaN,NaN,NaN
7594,NaN,NaN,NaN,NaN,NaN
7595,NaN,Data from database: Wealth Accounts,NaN,NaN,NaN


In [10]:
accountsdata = pd.read_csv("Wealth-AccountData.csv")

In [11]:
accountsdata.head(4)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19


In [12]:
accountsdata.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [13]:
accountsdata_clean = accountsdata.drop(['1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]','2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]','2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]','2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2016 [YR2016]', '2017 [YR2017]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]','2015 [YR2015]'], axis=1)

In [14]:
accountsdata_clean.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1999 [YR1999],2003 [YR2003],2007 [YR2007],2011 [YR2011],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.22E+10,5.21E+10,5.940000e+10,6.850000e+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13573.26,17132.03,1.999908e+04,2.359058e+04,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3259.893,2016.02,2.465247e+03,2.848523e+03,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6492.73,10641.06,1.308602e+04,1.579854e+04,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4536.972,2728.79,3.170332e+03,3.603694e+03,4161.137


In [15]:
accountsdata_clean.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1999 [YR1999]', '2003 [YR2003]', '2007 [YR2007]',
       '2011 [YR2011]', '2018 [YR2018]'],
      dtype='object')

In [16]:
accountsdata_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   1995 [YR1995]  7592 non-null   object 
 5   1999 [YR1999]  7592 non-null   object 
 6   2003 [YR2003]  7592 non-null   object 
 7   2007 [YR2007]  7592 non-null   float64
 8   2011 [YR2011]  7592 non-null   float64
 9   2018 [YR2018]  7592 non-null   object 
dtypes: float64(2), object(8)
memory usage: 593.3+ KB


In [17]:
non_numeric_rows = accountsdata_clean[pd.to_numeric(accountsdata_clean['1995 [YR1995]'], errors='coerce').isna()]

# Get the indices of non-numeric values
indices_of_non_numeric = non_numeric_rows.index

# Print or use the indices as needed
print(indices_of_non_numeric )
for i in indices_of_non_numeric:
    print(non_numeric_rows['1995 [YR1995]'][i])

Index([1224, 1239, 6369, 6384, 7409, 7410, 7412, 7424, 7425, 7427], dtype='int64')
..
..
..
..
..
..
..
..
..
..


In [18]:
accountsdata_clean['1995 [YR1995]'].replace('..', 0.0, inplace=True)

accountsdata_clean['1995 [YR1995]'] = accountsdata_clean['1995 [YR1995]'].astype(float)

In [19]:
non_numeric_rows = accountsdata_clean[pd.to_numeric(accountsdata_clean['1999 [YR1999]'], errors='coerce').isna()]

# Get the indices of non-numeric values
indices_of_non_numeric = non_numeric_rows.index

# Print or use the indices as needed
print(indices_of_non_numeric )
for i in indices_of_non_numeric:
    print(non_numeric_rows['1999 [YR1999]'][i])

Index([1224, 1239, 6369, 6384], dtype='int64')
..
..
..
..


In [20]:
accountsdata_clean['1999 [YR1999]'].replace('..', 0.0, inplace=True)

# Convert the column to a numeric type
accountsdata_clean['1999 [YR1999]'] = accountsdata_clean['1999 [YR1999]'].astype(float)

In [21]:
non_numeric_rows = accountsdata_clean[pd.to_numeric(accountsdata_clean['2003 [YR2003]'], errors='coerce').isna()]

# Get the indices of non-numeric values
indices_of_non_numeric = non_numeric_rows.index

# Print or use the indices as needed
print(indices_of_non_numeric )
for i in indices_of_non_numeric:
    print(non_numeric_rows['2003 [YR2003]'][i])

Index([1482, 1497], dtype='int64')
..
..


In [22]:
accountsdata_clean['2003 [YR2003]'].replace('..', 0.0, inplace=True)

# Convert the column to a numeric type
accountsdata_clean['2003 [YR2003]'] = accountsdata_clean['2003 [YR2003]'].astype(float)

In [23]:
non_numeric_rows = accountsdata_clean[pd.to_numeric(accountsdata_clean['2018 [YR2018]'], errors='coerce').isna()]

# Get the indices of non-numeric values
indices_of_non_numeric = non_numeric_rows.index

# Print or use the indices as needed
print(indices_of_non_numeric )
for i in indices_of_non_numeric:
    print(non_numeric_rows['2018 [YR2018]'][i])

Index([ 546,  561,  910,  925,  962,  977, 1170, 1185, 1222, 1237, 1430, 1445,
       1846, 1861, 2158, 2173, 2678, 2693, 2730, 2745, 3874, 3889, 3926, 3941,
       3953, 3954, 3955, 3956, 3957, 3958, 3959, 3967, 3968, 3969, 3970, 3971,
       3972, 3973, 3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982, 3999,
       4001, 4003, 4082, 4097, 4134, 4149, 4238, 4253, 4290, 4305, 4394, 4409,
       5070, 5085, 5798, 5813, 5954, 5969, 6162, 6177, 6682, 6697, 6994, 7009,
       7410, 7425],
      dtype='int64')
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..


In [24]:
accountsdata_clean['2018 [YR2018]'].replace('..', 0.0, inplace=True)

# Convert the column to a numeric type
accountsdata_clean['2018 [YR2018]'] = accountsdata_clean['2018 [YR2018]'].astype(float)

In [25]:
accountSeries = pd.read_csv("Wealth-AccountSeries.csv")

In [26]:
accountSeries.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [27]:
accountSeries_clean = accountSeries[['Code','Indicator Name','Topic','Long definition']]

In [28]:
cursor,connection = create_database()

In [29]:
accounts_country_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
                            Code VARCHAR,
                            Long_Name VARCHAR,
                            Short_Name VARCHAR,
                            Currency_Unit VARCHAR,
                            Table_Name VARCHAR
                           )
                            """)

In [30]:
cursor.execute(accounts_country_create)

In [31]:
connection.commit()

In [32]:
accounts_data_create = ("""CREATE TABLE IF NOT EXISTS accountsdata
                        (Country_Name VARCHAR,
                        Country_Code VARCHAR,
                        Series_Name  VARCHAR,
                        Series_Code VARCHAR,
                        year_1999  numeric,
                        year_1995  numeric,
                        year_2003  numeric,
                        year_2007  numeric,
                        year_2011  numeric,
                        year_2018  numeric)
                        """)

In [33]:
cursor.execute(accounts_data_create)
connection.commit()

In [34]:
account_series_create = (""" CREATE TABLE IF NOT EXISTS accountseries(
                        Code VARCHAR,
                        Indicator_Name VARCHAR,
                        Topic VARCHAR,
                        Long_definition VARCHAR
)
""")

In [35]:
cursor.execute(account_series_create)
connection.commit()


In [36]:
accounts_country_insert = ("""INSERT INTO accountscountry(                                              
                            Code ,
                            Long_Name ,
                            Currency_Unit ,
                            Table_Name,
                            Short_Name 
 )
                           VALUES (%s,%s,%s,%s,%s)
                                                      

""")

In [37]:
for i, row in accountscountry_clean.iterrows():
    cursor.execute(accounts_country_insert, list(row))

In [38]:
connection.commit()

In [39]:
accounts_data_insert = ("""INSERT INTO accountsdata
                        (Country_Name ,
                        Country_Code ,
                        Series_Name  ,
                        Series_Code ,
                        year_1995  ,
                        year_1999  ,
                        year_2003  ,
                        year_2007  ,
                        year_2011  ,
                        year_2018  )
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                        """)

In [40]:
for i, row in accountsdata_clean.iterrows():
    cursor.execute(accounts_data_insert,list(row))

In [41]:
connection.commit()


In [42]:
account_series_insert =(""" INSERT INTO accountseries
                        (Code,
                        Indicator_Name,
                        Topic,
                        Long_definition)
                        VALUES (%s,%s,%s,%s)
"""
                        

)

In [43]:
for i, row in accountSeries_clean.iterrows():
    cursor.execute(account_series_insert,list(row))